<a href="https://colab.research.google.com/github/arunangshudutta/DA6401_assignment3/blob/main/Ques_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls /content/drive/MyDrive/Colab\ Notebooks/DA6401/assignment\ 3/dakshina_dataset_v1.0/hi/lexicons

hi.translit.sampled.dev.tsv   hi.translit.sampled.train.tsv
hi.translit.sampled.test.tsv


In [3]:
import shutil

# Source folder in Google Drive
src_folder = '/content/drive/MyDrive/Colab Notebooks/DA6401/assignment 3/dakshina_dataset_v1.0/hi/lexicons'

# Destination in Colab's local storage
dst_folder = '/content/hindi_data'

# Copy the entire folder
shutil.copytree(src_folder, dst_folder)

'/content/hindi_data'

In [4]:
import pandas as pd

# Load the data
df = pd.read_csv("hindi_data/hi.translit.sampled.train.tsv", sep="\t", header=None, names=["target", "input", "num"])

# Drop rows with NaNs (if any)
df = df.dropna()

# Limit for quick testing (optional)
# df = df.sample(500)

print(df)

          target       input  num
0             अं          an    3
1        अंकगणित    ankganit    3
2           अंकल       uncle    4
3          अंकुर       ankur    4
4         अंकुरण     ankuran    3
...          ...         ...  ...
44199  ह्वेनसांग  hiuentsang    1
44200  ह्वेनसांग  hsuantsang    1
44201  ह्वेनसांग    hyensang    1
44202  ह्वेनसांग    xuanzang    1
44203          ॐ          om    3

[44202 rows x 3 columns]


In [5]:
from collections import Counter

# Special tokens
PAD_token = "<pad>"
SOS_token = "<sos>"  # start of sequence
EOS_token = "<eos>"  # end of sequence

def build_vocab(sequences):
    vocab = set(char for seq in sequences for char in seq)
    vocab = [PAD_token, SOS_token, EOS_token] + sorted(vocab)
    char2idx = {c: i for i, c in enumerate(vocab)}
    idx2char = {i: c for c, i in char2idx.items()}
    return char2idx, idx2char

input_char2idx, input_idx2char = build_vocab(df["input"])
target_char2idx, target_idx2char = build_vocab(df["target"])

print(f"Input vocab size: {len(input_char2idx)}")
print(f"Target vocab size: {len(target_char2idx)}")

Input vocab size: 29
Target vocab size: 66


In [6]:
input_idx2char

{0: '<pad>',
 1: '<sos>',
 2: '<eos>',
 3: 'a',
 4: 'b',
 5: 'c',
 6: 'd',
 7: 'e',
 8: 'f',
 9: 'g',
 10: 'h',
 11: 'i',
 12: 'j',
 13: 'k',
 14: 'l',
 15: 'm',
 16: 'n',
 17: 'o',
 18: 'p',
 19: 'q',
 20: 'r',
 21: 's',
 22: 't',
 23: 'u',
 24: 'v',
 25: 'w',
 26: 'x',
 27: 'y',
 28: 'z'}

In [7]:
def encode_sequence(seq, char2idx, add_sos_eos=True):
    tokens = [char2idx[c] for c in seq]
    if add_sos_eos:
        tokens = [char2idx[SOS_token]] + tokens + [char2idx[EOS_token]]
    return tokens

input_sequences = [encode_sequence(seq, input_char2idx) for seq in df["input"]]
target_sequences = [encode_sequence(seq, target_char2idx) for seq in df["target"]]

In [8]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset

class TransliterationDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx]), torch.tensor(self.targets[idx])

def collate_fn(batch):
    inputs, targets = zip(*batch)
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=input_char2idx[PAD_token])
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=target_char2idx[PAD_token])
    return inputs_padded, targets_padded

train_dataset = TransliterationDataset(input_sequences, target_sequences)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [11]:
i, o = next(iter(train_dataloader))

In [12]:
i.shape

torch.Size([32, 14])

In [13]:
input_vocab_size = len(input_char2idx)

In [14]:
df_val = pd.read_csv("hindi_data/hi.translit.sampled.dev.tsv", sep="\t", header=None, names=["target", "input", "num"])

input_val = [encode_sequence(seq, input_char2idx) for seq in df_val["input"]]
target_val = [encode_sequence(seq, target_char2idx) for seq in df_val["target"]]

val_dataset = TransliterationDataset(input_val, target_val)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [15]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---- Flexible Encoder ----
class Encoder(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, num_layers=1, cell_type="RNN"):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, embed_dim)
        rnn_cell = {"RNN": nn.RNN, "LSTM": nn.LSTM, "GRU": nn.GRU}[cell_type]
        self.rnn = rnn_cell(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.cell_type = cell_type

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, hidden = self.rnn(embedded)
        return hidden


# ---- Flexible Decoder ----
class Decoder(nn.Module):
    def __init__(self, output_dim, embed_dim, hidden_dim, num_layers=1, cell_type="RNN"):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_dim, embed_dim)
        rnn_cell = {"RNN": nn.RNN, "LSTM": nn.LSTM, "GRU": nn.GRU}[cell_type]
        self.rnn = rnn_cell(embed_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.cell_type = cell_type

    def forward(self, input, hidden):
        input = input.unsqueeze(1)  # [B] -> [B,1]
        embedded = self.embedding(input)  # [B,1,E]
        output, hidden = self.rnn(embedded, hidden)
        prediction = self.fc_out(output.squeeze(1))  # [B, V]
        return prediction, hidden


In [16]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, sos_idx, eos_idx, cell_type="RNN"):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.sos_idx = sos_idx
        self.eos_idx = eos_idx
        self.cell_type = cell_type

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        B, T = trg.shape
        output_dim = self.decoder.fc_out.out_features
        outputs = torch.zeros(B, T, output_dim).to(device)

        hidden = self.encoder(src)
        input = torch.tensor([self.sos_idx] * B).to(device)

        for t in range(T):
            output, hidden = self.decoder(input, hidden)
            outputs[:, t, :] = output

            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            input = trg[:, t] if teacher_force else output.argmax(1)

        return outputs


In [ ]:
import torch.optim as optim
from tqdm import tqdm

# Hyperparameters
embed_size = 64
hidden_size = 128
num_layers = 2
cell_type = "LSTM"  # or "RNN", "GRU"

# Initialize model
encoder = Encoder(len(input_char2idx), embed_size, hidden_size, num_layers, cell_type).to(device)
decoder = Decoder(len(target_char2idx), embed_size, hidden_size, num_layers, cell_type).to(device)
model = Seq2Seq(encoder, decoder, sos_idx=target_char2idx['<sos>'], eos_idx=target_char2idx['<eos>'], cell_type=cell_type).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=target_char2idx["<pad>"])

# Training Loop
for epoch in range(1, 11):
    model.train()
    epoch_loss = 0
    for src, trg in tqdm(train_dataloader):
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()

        output = model(src, trg)  # output: (B, T, V)
        output = output.view(-1, len(target_char2idx))
        trg = trg.view(-1)

        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # --- Validation ---
    model.eval()
    total_tokens = 0
    correct_tokens = 0
    with torch.no_grad():
        for val_src, val_trg in val_dataloader:
            val_src, val_trg = val_src.to(device), val_trg.to(device)
            val_output = model(val_src, val_trg)  # shape: (B, T, V)
            val_pred = val_output.argmax(dim=-1)  # (B, T)

            mask = val_trg != target_char2idx["<pad>"]
            correct = (val_pred == val_trg) & mask
            correct_tokens += correct.sum().item()
            total_tokens += mask.sum().item()

    val_accuracy = correct_tokens / total_tokens * 100

    print(f"Epoch {epoch} | Train Loss: {epoch_loss / len(train_dataloader):.4f} | Val Acc: {val_accuracy:.2f}%")


Epoch 1 | Train Loss: 2.1553 | Val Acc: 61.42%
Epoch 2 | Train Loss: 1.0523 | Val Acc: 72.44%
Epoch 3 | Train Loss: 0.7828 | Val Acc: 76.71%
